In [2]:
#Reuse data extraction modules from https://github.com/puyanoii/Twitter_Search
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
from ast import literal_eval

def unnest_json(dataframe, column):
    dataframe_new = json_normalize(dataframe[column].apply(literal_eval))
    return dataframe_new

In [3]:
#London step 2 & 3
# Accessing the Academic API 
def connect_to_endpoint(bearer_token, query, next_token=None):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    params = {
        'expansions' : "author_id,referenced_tweets.id,geo.place_id,in_reply_to_user_id,referenced_tweets.id.author_id",
        'tweet.fields' : "attachments,author_id,context_annotations,created_at,entities,public_metrics",
        'user.fields' : "created_at,username,verified,description,entities,id,location,name,public_metrics,url",
        'place.fields' : "contained_within,country,country_code,full_name,geo,id,name,place_type"}
#The endpoint
    if (next_token is not None):
        #specify the timeline
        url = "https://api.twitter.com/2/tweets/search/all?max_results=100&query={}&start_time=2021-03-08T00:00:00Z&end_time=2021-07-18T23:59:59.000Z&next_token={}".format(query, next_token)
    else:
        url = "https://api.twitter.com/2/tweets/search/all?max_results=100&start_time=2021-03-08T00:00:00Z&end_time=2021-07-18T23:59:59.000Z&query={}".format(query)
    response = requests.request("GET", url, params=params, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()
count = 0
flag = True
bearer_token = "my tokens"
data_collect = pd.DataFrame()
while flag:
    # Replace the count below with the number of Tweets you want to stop at. 
    # Note: running without the count check will result in getting more Tweets
    # that will count towards the Tweet cap
    if count >= 2000:
        break
    json_response = connect_to_endpoint(bearer_token, '("covid" OR "corona" OR "COVID-19") lang:en -is:retweet -is:reply has:geo place:London') #query
    result_count = json_response['meta']['result_count']
    while 'next_token' in json_response['meta']:
        next_token = json_response['meta']['next_token']
        #print(next_token)
        if result_count is not None and result_count > 0 and next_token is not None:
            df_tweet = pd.DataFrame(json_response['data'])
            df_user = pd.DataFrame(json_response['includes']['users'])
            df_places = pd.DataFrame(json_response['includes']['places'])
            df_full = pd.merge(df_tweet, df_user, how = 'left', left_on = 'author_id', right_on = 'id', suffixes=('_tweet', '_user'))
            df_full['geo'] = df_full['geo'].fillna({i: {} for i in df_full.index})
            df_full['geo_id'] =  df_full['geo'].apply(lambda x: x.get('place_id'))
            df_full = pd.merge(df_full, df_places, how = 'outer', left_on = 'geo_id', right_on = 'id', suffixes=('_meta', '_geo'))
            data_collect = data_collect.append(df_full)
            count += result_count
            print("{} tweets saved in the file".format(count))
           #json_response = connect_to_endpoint(bearer_token, '("covid" OR "corona") lang:en -is:retweet -is:reply has:geo place_country:GB', next_token)
            json_response = connect_to_endpoint(bearer_token, '("covid" OR "corona" OR "COVID-19") lang:en -is:retweet -is:reply has:geo place:London', next_token)#change city name here, and coutry name is GB
    else:
        flag = False
print("Finished searching for tweets!Total Tweet IDs saved: {}".format(count))
data_collect.to_csv("London(step 2 & 3)/London-step2&3.csv", encoding="utf-8",index=False, escapechar="\r")

100 tweets saved in the file
200 tweets saved in the file
300 tweets saved in the file
400 tweets saved in the file
500 tweets saved in the file
600 tweets saved in the file
700 tweets saved in the file
800 tweets saved in the file
900 tweets saved in the file
1000 tweets saved in the file
1100 tweets saved in the file
1200 tweets saved in the file
1300 tweets saved in the file
1400 tweets saved in the file
1500 tweets saved in the file
1600 tweets saved in the file
1700 tweets saved in the file
1800 tweets saved in the file
1900 tweets saved in the file
2000 tweets saved in the file
2100 tweets saved in the file
2200 tweets saved in the file
2300 tweets saved in the file
2400 tweets saved in the file
2500 tweets saved in the file
2600 tweets saved in the file
2700 tweets saved in the file
2800 tweets saved in the file
2900 tweets saved in the file
3000 tweets saved in the file
3100 tweets saved in the file
3200 tweets saved in the file
3300 tweets saved in the file
3400 tweets saved i